# Land Carbon Sink Analysis

Main notebook for the Land Carbon Sink Analysis project.

In [1]:
import xarray as xr
import matplotlib
import xesmf as xe
import numpy as np
import pandas as pd
import polars as pl
import os, sys

In [2]:
# directories
rawDir = "../data/raw/"
interimDir = "../data/interim/"
processedDir = "../data/processed/"

## Data import


In [3]:
# Environmental Contribution to land use change! NOT ELUC
# ELUC = xr.open_dataset(rawDir + "landUseChangeDorgeist.nc")

# Natural Land Sink (0.5 arcdegrees)
SLAND = xr.open_dataset(rawDir + "landSinkDorgeist.nc")

# Country mask (0.1 arcdegrees)
countryMask = xr.open_dataset(
    rawDir + "Country-mask_UNFCCC_layers-all-countries_grid-3600x1800.nc"
)

country_codes = pd.read_excel(rawDir + "Country-codes_UNFCCC_199countries.xlsx")
country_codes["Numeric"] = country_codes["Numeric"].astype(int)

# Get list of country codes
ctrs_UNFCCC = country_codes["Alpha-3"]

## Natural Land Sink Country Attribution

### Regrid countryMask to Match natLandSink Resolution

In [4]:
# Regrid `countryMask` to match `SLAND` coordinates
# regridder = xe.Regridder(countryMask, SLAND, method="conservative")
# countryMask_regridded_dor = regridder(countryMask)
# countryMask_regridded_dor.to_netcdf(interimDir + "countryMask_regridded_dor.nc")

In [5]:
# Load datasets
countryMask_regridded_dor = xr.open_dataset(interimDir + "countryMask_regridded_dor.nc")

# Inspect the coordinates
print(SLAND.coords)
print(countryMask_regridded_dor.coords)

Coordinates:
  * lat      (lat) float32 3kB 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon      (lon) float32 6kB -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
Coordinates:
  * ISOcode  (ISOcode) float64 2kB 4.0 24.0 8.0 ... 716.0 9.999e+03 5.555e+03
  * lat      (lat) float32 3kB 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon      (lon) float32 6kB -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9


## Natural Land Sink

### Check Regridding

In [6]:
# Get lat and lon names
if "latitude" in SLAND.dims:
    lat_name, lon_name = "latitude", "longitude"
else:
    lat_name, lon_name = "lat", "lon"

# Check that model grid and country grid agree
check_lat1 = np.max(
    np.abs(SLAND[lat_name].values - countryMask_regridded_dor[lat_name].values)
)
check_lon1 = np.max(
    np.abs(SLAND[lon_name].values - countryMask_regridded_dor[lon_name].values)
)
if check_lat1 > 0.001 or check_lon1 > 0.001:
    sys.exit("Coordinates do not agree")

# Re-index if there are small deviations in lat and lon
if (check_lat1 != 0) or (check_lon1 != 0):
    countryMask_regridded_dor = countryMask_regridded_dor.reindex(
        {lat_name: SLAND[lat_name], lon_name: SLAND[lon_name]},
        method="nearest",
    )

### Unit Conversion

In [7]:
# help(xe.util.cell_area)
# Compute grid areas (in km^2) for natSLAND


grid_areas_km2 = xe.util.cell_area(SLAND, earth_radius=6371.0)  # Earth radius in km


# Convert kgC/ha/year to GtC/year (global scale)


# Scale factor calculation


SLAND = SLAND.SLAND_trans * 100 * 1e-12 * grid_areas_km2


SLAND = SLAND.fillna(0)

### Country Attribution

In [8]:
# Dictionary for storing country data
data_ctrs_dor = dict()

# Loop over all country codes
for i, iso_alpha3 in enumerate(ctrs_UNFCCC):

    if np.mod(i, 20) == 0:
        print(f"  - processing country {i + 1} of {len(ctrs_UNFCCC)}")

    # Get numeric ISO code for the country
    iso_numeric = country_codes["Numeric"][
        country_codes["Alpha-3"] == iso_alpha3
    ].values[0]

    # Extract country mask for the current country
    mask = countryMask_regridded_dor.sel(ISOcode=iso_numeric)

    # Weight by fraction of country-specific land area
    weights = mask.land_fraction / mask.land_fraction_global

    # Sum up the natural land sink for the selected country
    data_sel = (SLAND * weights).sum((lat_name, lon_name))

    # Store results in dictionary
    data_ctrs_dor[iso_alpha3] = data_sel.values

  - processing country 1 of 199
  - processing country 21 of 199
  - processing country 41 of 199
  - processing country 61 of 199
  - processing country 81 of 199
  - processing country 101 of 199
  - processing country 121 of 199
  - processing country 141 of 199
  - processing country 161 of 199
  - processing country 181 of 199


In [9]:
##### Add fluxes in ocean grid cells to closest county ####

# Select only ocean grid cells that have non-zero land sink values
data_ocean = SLAND.where(countryMask_regridded_dor.land_fraction_global == 0)
data_ocean = data_ocean.where(data_ocean != 0)

# Get 2D arrays with latitudes and longitudes and keep only country areas
LON, LAT = np.meshgrid(countryMask_regridded_dor.lon, countryMask_regridded_dor.lat)

# Apply the mask to latitude and longitude arrays
LAT[countryMask_regridded_dor.land_fraction_global == 0] = np.nan
LON[countryMask_regridded_dor.land_fraction_global == 0] = np.nan


# Convert latitudes and longitudes to radians for distance calculation
LAT_rad = LAT * np.pi / 180
LON_rad = LON * np.pi / 180

In [10]:
# Loop over all grid cells to assign ocean grid values to closest country cells
for i_lat in range(0, len(data_ocean.lat)):
    for i_lon in range(0, len(data_ocean.lon)):

        # Read land sink data for the specific grid cell
        data_sel = data_ocean.isel(lat=i_lat, lon=i_lon)

        # Only process ocean grid cells with non-zero land sink values
        if not np.isnan(data_sel):

            # Get latitude and longitude of the ocean grid cell and convert to radians
            lat_ocean = data_sel.lat.item() * np.pi / 180
            lon_ocean = data_sel.lon.item() * np.pi / 180

            # Calculate the argument for arccos
            argument = np.sin(lat_ocean) * np.sin(LAT_rad) + np.cos(lat_ocean) * np.cos(
                LAT_rad
            ) * np.cos(lon_ocean - LON_rad)
            argument = np.clip(argument, -1, 1)  # Clip to avoid invalid values

            # Calculate distances
            distance = np.arccos(argument)

            # Find the closest land grid cell(s)
            index_closest = np.argwhere(distance == np.nanmin(distance))

            # Loop over closest grid cells and add values to the corresponding country
            for ind in index_closest:

                # Get latitude and longitude of the closest grid cell
                lat_sel = LAT[ind[0], ind[1]]
                lon_sel = LON[ind[0], ind[1]]

                # Select the land_fraction at the specified lat, lon
                land_fraction_at_point = countryMask_regridded_dor.sel(
                    lat=lat_sel, lon=lon_sel
                ).land_fraction

                # Find the index where land_fraction is non-zero
                non_zero_index = np.where(land_fraction_at_point.values != 0)[0]

                # There should only be one non-zero value, retrieve the corresponding ISOcode
                if len(non_zero_index) == 1:
                    iso_code = countryMask_regridded_dor.ISOcode[
                        non_zero_index[0]
                    ].values
                    print(f"ISO Code for the given lat, lon: {iso_code}")
                else:
                    print("No or multiple countries found for this lat, lon")

                # Retrieve country name for the ISO code
                ctr_name = country_codes["Alpha-3"][
                    country_codes["Numeric"] == iso_code
                ].values[0]

                # Add the ocean grid cell's natural land sink value to the closest country's data
                data_ctrs_dor[ctr_name] += data_sel.values / len(index_closest)

ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0
ISO Code for the given lat, lon: 795.0


### Save results

In [17]:
# Convert data_ctrs to a DataFrame with country names as the index
data_ctrs_dor_df = pd.DataFrame(
    list(data_ctrs_dor.items()), columns=["iso", "SLAND_dor"]
)

# Sort the DataFrame by 'Country' column if needed
data_ctrs_dor_df = data_ctrs_dor_df.sort_values(
    "SLAND_dor", ascending=False
).reset_index(drop=True)

# Add units to the DataFrame and save to Excel and CSV
data_ctrs_dor_df.to_excel(processedDir + "land_sink_by_country_dor.xlsx", index=False)

data_ctrs_dor_df.to_csv(processedDir + "land_sink_by_country_dor.csv", index=False)

### Check Total sum and country level sum

In [18]:
total_land_sink_Gt_per_yr = data_ctrs_dor_df["SLAND_dor"].sum()
Sland_sum = (SLAND).sum(dim=["lat", "lon"]).item()
print(
    "Country Sum Land Sink (Gt per yr) conservative method:", total_land_sink_Gt_per_yr
)

print("Total Grid Cell Sum Land Sink (Gt per yr) :", Sland_sum)

Country Sum Land Sink (Gt per yr) conservative method: -2.9931169301458542
Total Grid Cell Sum Land Sink (Gt per yr) : -2.9931169301458533
